In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
import pyspark.sql.types as T

In [3]:
spark = SparkSession.builder.master('local').appName('app').getOrCreate()

In [4]:
sc = spark.sparkContext

In [11]:
raw = sc.textFile("file:///home/alan/spark/guided-capstone/part-00000-092ec1db-39ab-4079-9580-f7c7b516a283-c000.txt")

In [12]:
parsed = raw.map(lambda line: parse_json(line)) 

In [13]:
x = parsed.take(10)
x

# raw.collect()[0]

[['2020-08-06',
  'Q',
  'SYMA',
  'NASDAQ',
  '2020-08-06 09:38:08.093',
  1,
  '2020-08-06 09:30:00.000',
  '',
  78.13370587077013,
  100,
  79.8251633824899,
  100,
  'Q'],
 ['2020-08-06',
  'Q',
  'SYMA',
  'NASDAQ',
  '2020-08-06 09:46:05.163',
  2,
  '2020-08-06 09:30:00.000',
  '',
  76.52304470696788,
  100,
  76.57240785476783,
  100,
  'Q'],
 ['2020-08-06',
  'Q',
  'SYMA',
  'NASDAQ',
  '2020-08-06 09:52:14.798',
  3,
  '2020-08-06 09:30:00.000',
  '',
  78.74535582037817,
  100,
  79.09279949812425,
  100,
  'Q'],
 ['2020-08-06',
  'Q',
  'SYMA',
  'NASDAQ',
  '2020-08-06 09:58:51.806',
  4,
  '2020-08-06 09:30:00.000',
  '',
  75.61362753973539,
  100,
  76.94977735069746,
  100,
  'Q'],
 ['2020-08-06',
  'Q',
  'SYMA',
  'NASDAQ',
  '2020-08-06 10:07:40.796',
  5,
  '2020-08-06 09:30:00.000',
  '',
  77.45084535147971,
  100,
  78.72533317224415,
  100,
  'Q'],
 ['2020-08-06',
  'Q',
  'SYMA',
  'NASDAQ',
  '2020-08-06 10:15:34.939',
  6,
  '2020-08-06 09:30:00.000',
  '

In [36]:
struct1 = StructType([StructField("trade_dt", T.DateType(), False),
                      StructField("rec_type", T.StringType(), False),
                      StructField("symbol",   T.StringType(), False),
                      StructField("exchange", T.StringType(), False),
                      StructField("event_tm", T.TimestampType(), False),
                      StructField("event_seq_nb", T.IntegerType(), False),
                      StructField("arrival_tm",   T.TimestampType(), False),
                      StructField("trade_pr", T.DecimalType(), True),
                      StructField("bid_pr",   T.DecimalType(), False),
                      StructField("bid_size", T.IntegerType(), False),
                      StructField("ask_pr",   T.DecimalType(), False),
                      StructField("ask_size", T.IntegerType(), False),
                      StructField("partition", T.StringType(), False)]
                    )

In [14]:
struct1 = StructType([StructField("trade_dt", T.StringType(), False),
                      StructField("rec_type", T.StringType(), False),
                      StructField("symbol",   T.StringType(), False),
                      StructField("exchange", T.StringType(), False),
                      StructField("event_tm", T.StringType(), False),
                      StructField("event_seq_nb", T.IntegerType(), False),
                      StructField("arrival_tm",   T.StringType(), False),
                      StructField("trade_pr", T.DecimalType(), True),
                      StructField("bid_pr",   T.DecimalType(), False),
                      StructField("bid_size", T.IntegerType(), False),
                      StructField("ask_pr",   T.DecimalType(), False),
                      StructField("ask_size", T.IntegerType(), False),
                      StructField("partition", T.StringType(), False)]
                    )

In [37]:
len(struct1)

13

In [15]:
data = spark.createDataFrame(parsed, schema=struct1)

In [16]:
data.show(4)

Py4JJavaError: An error occurred while calling o102.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (10.0.2.15 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/alan/.local/share/virtualenvs/spark-Gr4aDBKY/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/alan/.local/share/virtualenvs/spark-Gr4aDBKY/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/alan/.local/share/virtualenvs/spark-Gr4aDBKY/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/alan/.local/share/virtualenvs/spark-Gr4aDBKY/lib/python3.8/s

In [32]:
 y = parsed.map(lambda x : [1 for _ in x])

In [33]:
y.collect()

[[1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 

In [40]:
out = raw.collect()
print(out[0])

{"trade_dt":"2020-08-06","file_tm":"2020-08-06 09:30:00.000","event_type":"Q","symbol":"SYMA","event_tm":"2020-08-06 09:38:08.093","event_seq_nb":1,"exchange":"NASDAQ","bid_pr":78.13370587077013,"bid_size":100,"ask_pr":79.8251633824899,"ask_size":100}


In [9]:
import json

def parse_json(line : str):
    record = json.loads(line)
    record_type = record['event_type']
    record_keys = set(record.keys())
    
    def translate(s):
                      
        translations = {'event_type' : 'rec_type', 'file_tm' : 'arrival_tm', 'price' : 'trade_pr'}
        return s if s not in translations else translations[s]
    
    quote_columns = {'trade_dt', 'file_tm', 'event_type', 'symbol', 'event_tm', 'event_seq_nb',
                    'exchange', 'bid_pr', 'bid_size', 'ask_pr', 'ask_size'}
    
    trade_columns = {'trade_dt', 'file_tm', 'event_type', 'symbol', 'event_tm', 'event_seq_nb',
                    'exchange', 'price', 'size', 'execution_id'}
    
    is_valid_record = (record_type == 'Q' and record_keys == quote_columns) or \
                      (record_type == 'T' and record_keys == trade_columns)
   
    if is_valid_record:
       translated_record =  {translate(k):v for k,v in record.items()}  
       translated_record['partition'] = record_type                
       return get_common_event(translated_record)                      
    else:
       translated_record['partition'] = 'B' 
       translated_record['line'] = line              
       return get_common_event(translated_record)
    
    


In [20]:
for k in d.keys():
    print(k, d[k])

trade_dt 2020-08-06
file_tm 2020-08-06 09:30:00.000
event_type Q
symbol SYMA
event_tm 2020-08-06 09:38:08.093
event_seq_nb 1
exchange NASDAQ
bid_pr 78.13370587077013
bid_size 100
ask_pr 79.8251633824899
ask_size 100


In [10]:
def get_common_event(record : dict ):
    
    common_data = ('trade_dt', 'rec_type', 'symbol', 'exchange', 'event_tm', 'event_seq_nb','arrival_tm', 
                   'trade_pr',  'bid_pr', 'bid_size', 'ask_pr', 'ask_size', 'partition')
    
    return [record.get(k,"") for k in common_data] 
    
    